In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import pytz
from timezonefinder import TimezoneFinder
from astral.sun import sun
from astral import LocationInfo
import geocoder
from IPython.display import display

In [2]:
df = pd.read_csv('/Users/elliotyip/Desktop/Portfolio-project/Project-05/US-Accidents.csv')

In [5]:
display(df.shape)
display(df.columns)
display(df.dtypes)

(7728394, 46)

Index(['ID', 'Source', 'Severity', 'Start_Time', 'End_Time', 'Start_Lat',
       'Start_Lng', 'End_Lat', 'End_Lng', 'Distance(mi)', 'Description',
       'Street', 'City', 'County', 'State', 'Zipcode', 'Country', 'Timezone',
       'Airport_Code', 'Weather_Timestamp', 'Temperature(F)', 'Wind_Chill(F)',
       'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Direction',
       'Wind_Speed(mph)', 'Precipitation(in)', 'Weather_Condition', 'Amenity',
       'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway',
       'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal',
       'Turning_Loop', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight'],
      dtype='object')

ID                        object
Source                    object
Severity                   int64
Start_Time                object
End_Time                  object
Start_Lat                float64
Start_Lng                float64
End_Lat                  float64
End_Lng                  float64
Distance(mi)             float64
Description               object
Street                    object
City                      object
County                    object
State                     object
Zipcode                   object
Country                   object
Timezone                  object
Airport_Code              object
Weather_Timestamp         object
Temperature(F)           float64
Wind_Chill(F)            float64
Humidity(%)              float64
Pressure(in)             float64
Visibility(mi)           float64
Wind_Direction            object
Wind_Speed(mph)          float64
Precipitation(in)        float64
Weather_Condition         object
Amenity                     bool
Bump      

In [6]:
df['ID'].is_unique

True

In [7]:
# Set the display format for floats

pd.set_option('display.float_format', lambda x: '%.3f' % x)  

df[[
    'Temperature(F)',
    'Wind_Chill(F)',
    'Humidity(%)',
    'Pressure(in)',
    'Visibility(mi)',
    'Wind_Speed(mph)',
    'Precipitation(in)'
]].describe()

,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Precipitation(in)
count,7564541.000,5729375.000,7554250.000,7587715.000,7551296.000,7157161.000,5524808.000
mean,61.663,58.251,64.831,29.539,9.090,7.685,0.008
std,19.014,22.390,22.821,1.006,2.688,5.425,0.110
min,-89.000,-89.000,1.000,0.000,0.000,0.000,0.000
25%,49.000,43.000,48.000,29.370,10.000,4.600,0.000
50%,64.000,62.000,67.000,29.860,10.000,7.000,0.000
75%,76.000,75.000,84.000,30.030,10.000,10.400,0.000
max,207.000,207.000,100.000,58.630,140.000,1087.000,36.470


In [8]:
df[['Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight']]

,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,Night,Night,Night,Night
1,Night,Night,Night,Day
2,Night,Night,Day,Day
3,Night,Day,Day,Day
4,Day,Day,Day,Day
...,...,...,...,...
7728389,Day,Day,Day,Day
7728390,Day,Day,Day,Day
7728391,Day,Day,Day,Day
7728392,Day,Day,Day,Day


- The four columns above determines whether the accident record is categorised as day or night accident. We will use the time of sunrise and sunset time of the local timezone to append either 'day' or 'night' to `sunrise_sunset` for our analysis and drop the other 3.
- We will also change `sunrise_sunset` to `day_night` so it is more intuitive.
- `Source` is the source of data, we won't use that for analysis.  
- `Airport_Code` tells us the the closest weather station to th accident location, not particularly useful.  
- `Weather_Timestamp` tells us the the latest datetime as to when the data was obtained, we could use that to extract the datetime information, but we already have `Start_Time` for that. So let's drop that too.
- `End_Lat` and `End_Lng` will not be used. We will use `Start_Lat` and `Start_Lng` instead.
- `Distance(mi)` is the length of the road affected by the accident, we won't be using that to get any insight
- We'll use `Start_time` to determine the time of accident and drop `End_time`
- `Zipcode` and `Street` will not be helpful, we will not be analysing at that level of granularity.

In [9]:
df = df.drop(columns=["Civil_Twilight", "Nautical_Twilight", 'Astronomical_Twilight', 'Source', 'Airport_Code', 'Weather_Timestamp', 'End_Lat', 'End_Lng', 'Distance(mi)', 'Zipcode', 'End_Time', 'Street']) 

In [10]:
df = df.rename(columns={'Sunrise_Sunset': 'Day_Night'})

We will also change the datetime column from string to datetime object, are some inconsistent string format so we will standardise it

In [11]:
# Number of rows with microseconds

print(df['Start_Time'].str.contains(r'\..+', regex=True).sum())

743166


In [12]:
# Removing microseconds

df['Start_Time'] = df['Start_Time'].replace(r'\..+', '', regex=True)

In [13]:
df['Start_Time'] = pd.to_datetime(df['Start_Time'])

In [14]:
print((df['Day_Night'].isna() & df['Timezone'].notna()).sum())
print((df['Day_Night'].isna() & df['Timezone'].isna()).sum())

23022
224


23022 rows does not have day/night, let's write a function to determine the sunrise/sunset time for these data points using timezone and pytz, then populate day_night column.

For the remaining 224 data points without timezone, we will use coordinates in the Start_Lat/Start_Lng column to determine the timezone, then populate day_night column.

In [15]:
# Getting timezone using coordinates
def get_timezone(lat, long):
    tf = TimezoneFinder()
    local_timezone_str = tf.timezone_at(lat=float(lat), lng=float(long))
    local_timezone = pytz.timezone(local_timezone_str)

    return local_timezone


# Get local sunrise time using timezone and date
def get_local_sunrise(timezone, date):
    # Convert 'date' to datetime object

    # Create LocationInfo for the given timezone
    timezone_info = LocationInfo(timezone)

    # Calculate sunrise time using astral
    s = sun(timezone_info.observer, date=date)
    local_sunrise = s['sunrise']

    return local_sunrise


# Assigning day/night using local sunrise time
def classify_day_night(row):

    time_column = row['Start_Time']

    local_sunrise_time = get_local_sunrise(row['Timezone'], row['Start_Time'])

    if time_column < local_sunrise_time.time():
        return 'Night'
    else:
        return 'Day'

In [16]:
# Assigning timezone to rows where Day_Night column is empty, using coordinates.

# mask = pd.isna(df['Day_Night']) & pd.isna(df['Timezone'])
# filtered_df = df[mask].copy()

# filtered_df['Timezone'] = filtered_df.apply(lambda row: get_timezone(row['Start_Lat'], row['Start_Lng']), axis=1)
# df.loc[mask, 'Timezone'] = filtered_df['Timezone']

In [17]:
print((df['Day_Night'].isna() & df['Timezone'].isna()).sum())

224


In [18]:
# Rows with no Day_Night values
df['Day_Night'].isna().sum()

23246

In [19]:
df3 = df[df['Day_Night'].isna()]

In [20]:
# Assigning day/night using sunrise time for each timezone

local_sunrise_time = df3.apply(
    lambda row: get_local_sunrise(row['Timezone'], row['Start_Time']), axis=1
)

# df.apply(lambda row: classify_sunrise_sunset(row))

local_sunrise_time

85968     2016-08-02 04:26:12.038734+00:00
111080    2016-06-01 03:48:47.377042+00:00
119772    2016-12-23 08:04:32.406150+00:00
122929    2017-01-17 07:57:11.153584+00:00
123702    2017-01-24 07:49:29.302526+00:00
                        ...               
7619724   2017-10-09 06:14:16.756532+00:00
7629988   2017-09-06 05:21:02.166878+00:00
7636150   2017-09-21 05:44:55.959788+00:00
7639338   2017-09-29 05:57:49.894653+00:00
7664113   2019-04-23 04:48:44.517983+00:00
Length: 23246, dtype: datetime64[ns, UTC]

In [21]:
local_sunrise_time = local_sunrise_time.dt.tz_localize(None)

df3['Day_Night'] = np.where(df3['Start_Time'] < local_sunrise_time, 'Night', 'Day')
df3['Day_Night'].value_counts()

/var/folders/3v/nvf5sb85235fjz6fgzjnl1gr0000gn/T/ipykernel_17496/12994331.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['Day_Night'] = np.where(df3['Start_Time'] < local_sunrise_time, 'Night', 'Day')


Day_Night
Day      20190
Night     3056
Name: count, dtype: int64

In [22]:
# Update df['Day_Night'] with data in df3['Day_Night']

df.loc[df3.index, 'Day_Night'] = df3['Day_Night']

In [23]:
df['Day_Night'].value_counts()

Day_Night
Day      5354743
Night    2373651
Name: count, dtype: int64

In [24]:
# Checking for other null values

df.columns[df.isnull().any()]

Index(['Description', 'City', 'Timezone', 'Temperature(F)', 'Wind_Chill(F)',
       'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Direction',
       'Wind_Speed(mph)', 'Precipitation(in)', 'Weather_Condition'],
      dtype='object')

In [25]:
empty_row_counts = df.isnull().sum()

# Create a DataFrame to display the results
result_df = pd.DataFrame({'Column': empty_row_counts.index, 'Empty_Row_Count': empty_row_counts.values})

# Print the result
print(result_df)


               Column  Empty_Row_Count
0                  ID                0
1            Severity                0
2          Start_Time                0
3           Start_Lat                0
4           Start_Lng                0
5         Description                5
6                City              253
7              County                0
8               State                0
9             Country                0
10           Timezone             7808
11     Temperature(F)           163853
12      Wind_Chill(F)          1999019
13        Humidity(%)           174144
14       Pressure(in)           140679
15     Visibility(mi)           177098
16     Wind_Direction           175206
17    Wind_Speed(mph)           571233
18  Precipitation(in)          2203586
19  Weather_Condition           173459
20            Amenity                0
21               Bump                0
22           Crossing                0
23           Give_Way                0
24           Junction    

- City will be quite important for our analysis, let's use `geocoder` library to fill in the empty cities row.  
- For most of our missing continuous data, we will use average method to fill them in but on a city level. Drier cities might have a significant difference in precipitation than wet ones.  
- Average `precipitation(in)`,`wind_speed(mph)`, `visibility`, `pressure`, `humidity`, `wind_chill`, `temperature` on City level.
- Standardise `wind direction` (Full name to short name).
- Add a column called `month` and populate the column with the text of month from `start_date``, this will allow us to breakdown the data by month and see weather seasonality has an effect on accidents rate.

In [26]:
# Defining function to find city using coordinate

def assign_city(lat, long):
    location = geocoder.osm([lat, long], method='reverse')
    city = location.city
    if city == None:
        return pd.NA
    else:
        return city

In [27]:
df['City'].isna().sum()
df_city_na = df[df['City'].isna()]

In [28]:
# Filling in cities

assigned_city = df_city_na.apply(
    lambda row: assign_city(row['Start_Lat'], row['Start_Lng']), axis=1
)

df.loc[df_city_na.index, 'City'] = df_city_na

In [29]:
# Filling in data for NA rows

columns_to_fill = [
    'Temperature(F)',
    'Wind_Chill(F)',
    'Humidity(%)',
    'Pressure(in)',
    'Visibility(mi)',
    'Wind_Speed(mph)',
    'Precipitation(in)'
]

for i in columns_to_fill:
    df[i] = df[i].fillna(df.groupby('City')[i].transform('mean'))

In [30]:
df['Wind_Direction'].value_counts()

direction_remap = {
    'South':'S',
    'North': 'N',
    'East': 'E',
    'West': 'W',
    'Variable': 'VAR',
    'Calm': 'CALM'

}

df['Wind_Direction'] = df['Wind_Direction'].replace(direction_remap)

wind_dir_list = [
    'CALM', 'S', 'W', 'N', 'SSW', 'E', 'WNW', 'NW', 'VAR', 'SW', 'WSW', 'SSE', 'NNW', 'SE', 'ESE', 'NE', 'ENE', 'NNE'
]

print(df['Wind_Direction'].value_counts())

Wind_Direction
CALM    1330181
S        597364
W        548664
N        460536
SSW      384840
E        382459
WNW      378781
NW       369352
VAR      364562
SW       364470
WSW      353806
SSE      349110
NNW      333427
SE       294901
ESE      268311
NE       258639
ENE      258474
NNE      255311
Name: count, dtype: int64


In [31]:
df.isnull().sum()

ID                        0
Severity                  0
Start_Time                0
Start_Lat                 0
Start_Lng                 0
Description               5
City                    253
County                    0
State                     0
Country                   0
Timezone               7808
Temperature(F)         7894
Wind_Chill(F)         23096
Humidity(%)            7896
Pressure(in)           7887
Visibility(mi)        17185
Wind_Direction       175206
Wind_Speed(mph)       20544
Precipitation(in)     52526
Weather_Condition    173459
Amenity                   0
Bump                      0
Crossing                  0
Give_Way                  0
Junction                  0
No_Exit                   0
Railway                   0
Roundabout                0
Station                   0
Stop                      0
Traffic_Calming           0
Traffic_Signal            0
Turning_Loop              0
Day_Night                 0
dtype: int64

There are still some rows with null data, this is because for those cities there are no values to be aggregated to begin with, we will drop these rows. We also drop rows where `City` has no data.

In [32]:
df = df.dropna(subset=[
    'Temperature(F)',
    'Wind_Chill(F)',
    'Humidity(%)',
    'Pressure(in)',
    'Visibility(mi)',
    'Wind_Direction',
    'Wind_Speed(mph)',
    'Precipitation(in)',
    'City',
    'Description'
]
)

df.isnull().sum()

ID                       0
Severity                 0
Start_Time               0
Start_Lat                0
Start_Lng                0
Description              0
City                     0
County                   0
State                    0
Country                  0
Timezone                 0
Temperature(F)           0
Wind_Chill(F)            0
Humidity(%)              0
Pressure(in)             0
Visibility(mi)           0
Wind_Direction           0
Wind_Speed(mph)          0
Precipitation(in)        0
Weather_Condition    34352
Amenity                  0
Bump                     0
Crossing                 0
Give_Way                 0
Junction                 0
No_Exit                  0
Railway                  0
Roundabout               0
Station                  0
Stop                     0
Traffic_Calming          0
Traffic_Signal           0
Turning_Loop             0
Day_Night                0
dtype: int64

In [33]:
# Adding 'month' column next to Start_Time

df.insert(df.columns.get_loc('Start_Time'), 'month', df['Start_Time'].dt.strftime('%B'))

In [34]:
df['month'].value_counts()

month
December     822551
November     736053
January      729090
October      656989
February     641546
September    631506
August       583017
April        572862
June         551788
May          542992
March        540070
July         495408
Name: count, dtype: int64

Finally let's find and drop duplicates

In [35]:
df = df.drop_duplicates()

In [36]:
df.shape

(7503872, 35)

In [37]:
df.to_csv('/Users/elliotyip/Desktop/Portfolio-project/Project-05/US-Accidents-final.csv', index=False)